In [3]:
from google.colab import drive
root = "/content/drive/"
drive.mount(root)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import json, os, sys, re, json

path = os.path.join(root, "My Drive/Colab Notebooks/COSE474")
os.makedirs(path, exist_ok=True)

od_path = os.path.join(path, "Rust_Code_Generation")
os.makedirs(od_path, exist_ok=True)

%cd "{od_path}"

/content/drive/My Drive/Colab Notebooks/COSE474/Rust_Code_Generation


In [2]:
pip install transformers==4.45.2

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [7]:
from huggingface_hub import login

with open("token_file.txt", "r") as file:
    token = file.read().strip()

# Log in using your token
login(token)

In [17]:
model_name = "meta-llama/CodeLlama-13b-Instruct-hf"
# model_name = "meta-llama/Llama-3.2-B-Instruct"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU (or CPU if no GPU available)
model = model.to(device)

RuntimeError: Bad StatusOr access: RESOURCE_EXHAUSTED: Error allocating device buffer: Attempting to allocate 36.00M. That was not possible. There are 14.23M free.; (0x0x0_HBM0)

In [18]:
device = next(model.parameters()).device
print(f"The model is loaded on: {device}")

The model is loaded on: cpu


In [19]:
from transformers import StoppingCriteria, StoppingCriteriaList, AutoModelForCausalLM, AutoTokenizer

class StopOnRUSTEND(StoppingCriteria):
    def __init__(self, stop_sequence: str, tokenizer, max_occurrences: int = 2):
        self.stop_sequence = stop_sequence
        self.tokenizer = tokenizer
        self.max_occurrences = max_occurrences
        self.current_count = 0

    def __call__(self, input_ids, scores, **kwargs):
        # Convert the current generated tokens back to text
        text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        # Count occurrences of 'RUST_END' in the generated text
        self.current_count = text.count(self.stop_sequence)

        # Stop if the 'RUST_END' sequence has appeared max_occurrences times
        return self.current_count >= self.max_occurrences

In [20]:
class RustCodeGenerator:
    def __init__(self, input_filename, output_filename, tokenizer, model, num_samples = 164, total_samples = 164, start_idx = 0, max_new_tokens = 300):
        self.input_filename = input_filename
        self.output_filename = output_filename
        self.tokenizer = tokenizer
        self.model = model
        self.num_samples = num_samples if num_samples <= total_samples else total_samples
        self.total_samples = total_samples
        self.start_idx = start_idx
        self.max_new_tokens = max_new_tokens

    def print_progress_bar(self, current, total, bar_length=50):
        # Calculate the percentage of completion
        percent = (current / total) * 100
        # Determine the number of "#" characters in the bar based on the percentage
        filled_length = int(bar_length * current // total)
        bar = '#' * filled_length + '-' * (bar_length - filled_length)

        # Use '\r' to overwrite the current line and display the loading bar
        sys.stdout.write(f'\rProgress: |{bar}| {percent:.2f}% ({current}/{total})')
        sys.stdout.flush()

    def process_line(self, line, row):
        prompt = (
            f"You are an expert Rust programmer. {row['instruction']}"
            "In your response, use RUST_BEGIN and RUST_END to delimit the rust source code\nRUST_BEGIN"
        )

        inputs = self.tokenizer(prompt, return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}

        stop_sequence = "RUST_END"
        stopping_criteria = StoppingCriteriaList([StopOnRUSTEND(stop_sequence, tokenizer)])

        outputs = self.model.generate(
            **inputs,
            num_return_sequences=1,
            #temperature=0.7,   # Sampling temperature; lower values make output more deterministic
            #top_p=0.9,         # Top-p sampling to keep only tokens with cumulative probability >= 0.9
            #do_sample=True,     # Enables sampling instead of greedy decoding
            max_new_tokens=self.max_new_tokens,
            stopping_criteria=stopping_criteria,
            return_dict_in_generate=True,
            output_scores=True
        )

        decoded_texts = self.tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)
        generated_code = "".join(decoded_texts)
        matches = re.findall(r'RUST_BEGIN\s*(.*?)\s*RUST_END', generated_code, re.DOTALL)

        rust_code = matches[1].strip() if len(matches) >= 2 else "None"
        return {
            'task_id': row["task_id"],
            'instruction': row["instruction"],
            'generated_code': rust_code,
            'test': row["test"]
        }

    def save_output(self, generated_codes):
        with open(self.output_filename, "w") as file:
            json.dump(generated_codes, file, indent=0)

    def process(self):
        generated_codes = []

        # Load existing data if the file exists
        if os.path.exists(self.output_filename) and os.path.getsize(self.output_filename) > 0:
            with open(self.output_filename, "r") as file:
                generated_codes = json.load(file)

        with open(self.input_filename) as f:
            for i in range(self.total_samples):
                if i < self.start_idx:
                    f.readline().strip()  # Skip lines until start_idx
                    continue

                line = f.readline().strip()
                self.print_progress_bar(i - self.start_idx, self.num_samples)
                row = json.loads(line)

                generated_code_data = self.process_line(line, row)
                generated_codes.append(generated_code_data)

                # Save progress to the output file after each iteration
                self.save_output(generated_codes)

                count = i - self.start_idx
                if count + 1 >= self.num_samples:
                    break
            self.print_progress_bar(self.num_samples, self.num_samples)


    def print_example_code(self, idx=0):

        if os.path.exists(self.output_filename) and os.path.getsize(self.output_filename) > 0:
            with open(self.output_filename, "r") as file:
                generated_codes = json.load(file)
                if generated_codes:
                    # Take the first entry
                    try:
                      generated_code_data = generated_codes[idx]
                    except IndexError:
                      print("Index out of range.")
                      return

                    task_id = generated_code_data.get("task_id", "No task available")
                    prompt = generated_code_data.get("instruction", "No prompt available")
                    code = generated_code_data.get("generated_code", "No code available")

                    print("Task " + task_id + ":")
                    print("Example Prompt:")
                    print(prompt)
                    print("\nGenerated Code:")
                    print(code)
                else:
                    print("No data available in the output file.")
        else:
            print("Output file does not exist or is empty.")


In [ ]:
input_filename = "humanevalpack.jsonl"
output_filename = "output.json"
num_samples = 1
start_idx = 89

generator = RustCodeGenerator(
    input_filename,
    output_filename,
    tokenizer,
    model,
    # num_samples=1,
    # start_idx=0,
    max_new_tokens = 400
)

generator.process()

Progress: |--------------------------------------------------| 0.00% (0/164)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Progress: |--------------------------------------------------| 0.61% (1/164)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Progress: |--------------------------------------------------| 1.22% (2/164)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
generator.print_example_code()

In [ ]:
generator.print_example_code(89)